In [1]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import torch
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization, Reshape, Conv2D, Conv2DTranspose, ReLU, LeakyReLU,InputLayer, UpSampling2D,ZeroPadding2D
import tensorflow.keras.datasets as dts 
from PIL import Image, ImageOps
from tensorflow.keras.utils import to_categorical
from google.colab.patches import cv2_imshow

In [3]:
y_train = np.concatenate((y_train,y_test))
x_train = np.concatenate((x_train,x_test))

In [4]:
from tensorflow.compat.v1 import keras
import tensorflow as tf

config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
sess = tf.compat.v1.Session(config=config) 
tf.compat.v1.keras.backend.set_session(sess)

In [5]:
x = (x_train.astype(np.float32) - 127.5) / 127.5
x = np.expand_dims(x,-1)

In [6]:
x.shape

(70000, 28, 28, 1)

In [7]:
y = to_categorical(y_train)

In [8]:
y.shape

(70000, 10)

In [9]:
lr = 0.0001
batch_size = 128
betas = (0.5, 0.999)
noise_size = 32

In [10]:
discriminator = keras.Sequential([
  InputLayer((28,28,1)),
  Conv2D(32, kernel_size=3, strides=2, padding="same"),
  LeakyReLU(alpha=0.2),
  Dropout(0.25),
  Conv2D(64, kernel_size=3, strides=2, padding="same"),
  ZeroPadding2D(padding=((0,1),(0,1))),
  BatchNormalization(momentum=0.8,),
  LeakyReLU(alpha=0.2),
  Dropout(0.25),
  Conv2D(256, kernel_size=3, strides=1, padding="same"),
  BatchNormalization(momentum=0.8),
  LeakyReLU(alpha=0.2),
  Dropout(0.25),
  Flatten(),
  Dense(10, activation='sigmoid'),
])

discriminator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 32)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 zero_padding2d (ZeroPadding  (None, 8, 8, 64)         0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 64)         256       
 ormalization)                                          

In [11]:
optimizer = keras.optimizers.Adam(lr,betas[0],betas[1])
discriminator.compile(loss='binary_crossentropy', 
  optimizer=optimizer)

In [12]:
generator = keras.Sequential([
  InputLayer((noise_size+10,)),
  Dense(42 * 7 * 7, activation="relu"),
  Reshape((7, 7, 42)),
  UpSampling2D(),
  Conv2D(42, kernel_size=3, padding="same"),
  BatchNormalization(momentum=0.8),
  ReLU(),
  UpSampling2D(),
  Conv2D(42, kernel_size=3, padding="same"),
  BatchNormalization(momentum=0.8),
  ReLU(),
  Conv2D(42, kernel_size=3, padding="same"),
  BatchNormalization(momentum=0.8),
  ReLU(),
  Conv2D(1, kernel_size=3, padding="same", activation='tanh')
])

generator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 2058)              88494     
                                                                 
 reshape (Reshape)           (None, 7, 7, 42)          0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 14, 14, 42)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 42)        15918     
                                                                 
 batch_normalization_2 (Batc  (None, 14, 14, 42)       168       
 hNormalization)                                                 
                                                                 
 re_lu (ReLU)                (None, 14, 14, 42)       

In [13]:
optimizer = keras.optimizers.Adam(lr,betas[0],betas[1])
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

In [14]:
optimizer = keras.optimizers.Adam(lr,betas[0],betas[1])
z = Input(shape=(noise_size+10,))
img = generator(z)
valid = discriminator(img)
combined = keras.Model(z, valid)

In [15]:
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [19]:
num_epochs = 2000

In [20]:
for epoch in range(num_epochs):  
  idx = np.random.randint(0,x_train.shape[0], batch_size)
  imgs = x[idx]
  setting = y[idx]
  noise = np.random.normal(0, 1, (batch_size, noise_size))
  settingNoise = np.concatenate((noise,setting),1)
  gen_imgs = generator.predict(settingNoise)

  d_loss_real = discriminator.train_on_batch(imgs, setting)
  d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((batch_size, 10)))
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  setting = to_categorical(np.random.randint(10,size=batch_size))
  noise = np.random.normal(0, 1, (batch_size, noise_size)) 
  settingNoise = np.concatenate((noise,setting),1)
  valid_y = setting

  discriminator.trainable = False
  g_loss = combined.train_on_batch(settingNoise, valid_y)
  discriminator.trainable = True
  
  print (epoch, d_loss, g_loss)

0 0.6524766981601715 0.5032910704612732
1 0.30962442606687546 0.36817532777786255
2 0.22792042419314384 0.383321613073349
3 0.21750978752970695 0.40154922008514404
4 0.2164226621389389 0.4019011855125427
5 0.20663408190011978 0.4017904996871948
6 0.21737881377339363 0.3655517101287842
7 0.20209715142846107 0.3930838406085968
8 0.20239774137735367 0.3717014789581299
9 0.1931314617395401 0.39399462938308716
10 0.19294746592640877 0.38359227776527405
11 0.18431373685598373 0.3697325885295868
12 0.19583269208669662 0.3899339437484741
13 0.1899196058511734 0.3803234398365021
14 0.1811884455382824 0.3810902237892151
15 0.17625100910663605 0.3760077953338623
16 0.1791192889213562 0.3920811414718628
17 0.16146013513207436 0.3975660800933838
18 0.16312668472528458 0.38476890325546265
19 0.16536490991711617 0.3941633701324463
20 0.16100870817899704 0.3825846314430237
21 0.16526076197624207 0.39618295431137085
22 0.16379248350858688 0.3832448720932007
23 0.15244999527931213 0.3889787197113037
24 

In [21]:
def genNum(num):
  noise = np.random.normal(0, 1, (1, noise_size))
  setting = to_categorical([num],10)
  settingNoise = np.concatenate((noise,setting),1)
  generated_samples = generator(settingNoise)
  generated_samples = np.array(generated_samples)
  generated_samples = ((generated_samples * 0.5) + 0.5) * 255
  generated_samples = generated_samples.round()
  generated_samples = generated_samples.astype(np.uint8)
  return generated_samples

In [58]:
sample = genNum(4)
cv2_imshow(sample[0])

In [59]:
discriminator.save('NUMdis_2000.h5')
generator.save('NUMgen_2000.h5')